<a href="https://colab.research.google.com/github/krish123-m/Krish/blob/main/SpamDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Install required libraries (run this first)
!pip install -q gradio scikit-learn torch pandas

# Import libraries
import gradio as gr
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# 1. Load sample data
data = {
    "text": [
        "Congratulations! You've won a $1000 Walmart gift card. Click to claim.",
        "Hi, can we meet for lunch tomorrow?",
        "You have been selected for a prize. Reply to win now!",
        "Let's schedule a meeting next week to discuss the project.",
        "Win money by clicking this link!",
        "Reminder: Your bill is due tomorrow.",
        "Earn cash fast by working from home!",
        "Hey, are you coming to the party tonight?",
    ],
    "label": [1, 0, 1, 0, 1, 0, 1, 0]  # 1=Spam, 0=Not Spam
}
df = pd.DataFrame(data)

# 2. Preprocessing: Convert text to numeric vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["text"]).toarray()
y = torch.tensor(df["label"]).float()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor = torch.from_numpy(X_test).float()

# 3. Define simple neural network model
class SpamClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

# Instantiate model, define loss and optimizer
model = SpamClassifier(X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 4. Train the model
for epoch in range(30):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor).squeeze()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        print(f"Epoch {epoch+1}/30, Loss: {loss.item():.4f}")

print(f"\nTraining complete. Final loss: {loss.item():.4f}")

# 5. Define prediction function for Gradio
def predict_spam(text):
    vec = vectorizer.transform([text]).toarray()
    tensor_input = torch.from_numpy(vec).float()
    with torch.no_grad():
        output = model(tensor_input)
        prob = output.item()
    return {"Spam": prob, "Not Spam": 1 - prob}

# 6. Gradio Interface
gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(lines=3, placeholder="Enter email text here..."),
    outputs=gr.Label(num_top_classes=2),
    title="📧 Simple Spam Detector (Neural Network)",
    description="A basic neural network model to detect spam emails. Enter a message to see its spam probability."
).launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 